# Estimating the phase of the Hydrogen molecule using Iterative Phase Estimation

## Setup

First, make sure that you have the latest version of Qiskit installed. To upgrade your Qiskit package, run the following command:

```bash
pip install --upgrade qiskit
```

Get an API key from IonQ. This will be used by the IonQ provider inside Qiskit to submit circuits to the IonQ platform.

After securing an API key, install the python package `qiskit_ionq` using `pip`:

```bash
pip install qiskit_ionq
```

(IonQ's adapter for Qiskit is currently in private beta -- your feedback is welcomed!)

### (Optional) Extra Dependencies

Some examples use additional Python dependencies; please make sure to `pip install` them as needed.

Dependencies:
* `matplotlib`: To run `qiskit.visualization.plot_histogram`.

**NOTE**: The provider expects an API key to be supplied via the `token` keyword argument to its constructor. If no token is directly provided, the provider will check for one in the `QISKIT_IONQ_API_TOKEN` environment variable.

Now that the Python package has been installed, you can import and instantiate the provider:

In [1]:
#import Aer here, before calling qiskit_ionq_provider
from qiskit import Aer

from qiskit_ionq import IonQProvider 

#Call provider and set token value
provider = IonQProvider(token='my token')

The `provider` instance can now be used to create and submit circuits to IonQ.

### Backend Types

The IonQ provider supports two backend types:
* `ionq_simulator`: IonQ's simulator backend.
* `ionq_qpu`: IonQ's QPU backend.

To view all current backend types, use the `.backends` property on the provider instance:

In [2]:
provider.backends()

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]

### Why do I care about the Iterative Phase Estimation Algorithm (IPEA)? What can you do with it? What is it?

Recall your linear algebra knowledge. More specifically, recall the concepts of eigenvalues and eigenvectors. Once you have these two notions solified, consider a unitary operator $U$ and a state $\left|\psi\right>$ such that the following relation is true : 

$$U\left|\psi\right>=e^{2\pi i\phi}\left|\psi\right>$$

In other words, $U$ has eigenvector $\left|\psi\right>$ with corresponding eigenvalue $e^{2\pi i\phi}$. $\phi$ is a real number, and it is our job to find out what its value is. That's it! That's the problem we are trying to solve. And we will solve it using this algorithm known as Iterative Phase Estimation.

### Okay, what is this algorithm and how do we use it to solve this problem?

At this point, I will provide a peripheral explanation of what the algorithm essentially does. The detailed explanation will follow as I actually code the algorithm. Before I give the basic summary though, keep the following point in mind - 

IMPORTANT : We assume that $\phi$, the value we are trying to find, can be expressed as $\frac{\phi_1}{2} + \frac{\phi_2}{4} + \frac{\phi_3}{8} + ... + \frac{\phi_n}{2^n}$. Here each of the $\phi_k$'s are either 0 or 1. Another way of thinking about this key assumption is that $\phi_1\phi_2\phi_3...\phi_n$ is the binary representation of $2^n\phi$ and $n$ is the number of iterations that we do 

Now for the general working : as the name of the algorithm suggests, there will be several iterations. Consider iteration k. The goal of this iteration will be to determine $\phi_{n-k+1}$ in the expression for $\phi$ above. To do this, the algorithm uses a circuit with two qubits : an auxiliary qubit and a main qubit (typically, this would be true. In this notebook we use three qubits because the problem we are tackling requires it). What will end up happening in every iteration is that $U$ will be acted on the main qubit in such a way that the auxiliary qubit's state, upon measurement, will collapse into $\phi_{n-k+1}$. Once we have all the $\phi_i$'s we can just put them into the above expression and solve for $\phi$.

### Okay, cool! What are we going to do with this algorithm? Where does hydrogen come in?

Okay, now we are going to put the IPEA to the test! In fact, we are going to do more than that. Here is the problem we are going to solve - 

While we know how to find the energy eigenvalues of the hydrogen atom (we know how to solve the Schrodinger equation exactly for this Hamiltonian), we don't have a way to exactly solve for the energy eigenvalues of the hydrogen molecule (we don't know how to exactly solve the Schrodinger equation for this other Hamiltonian). Using the IPEA, we are going to obtain a very good approximate solution! You will see just how good our approximation is at the very end of the notebook. We are going to solve for the ground-state energy of the hydrogen molecule.

How do we formulate this problem into an IPEA problem? Well, simple! The IPEA essentially tells you something about the eigenvalue of the unitary operator that you feed into it, right? So it comes down to smartly choosing the unitary operator that you use - if you feed in a unitary that, perhaps, has an eigenvalue related to the energy eigenvalue we wish to determine (remember that we wish to find the ground-state energy), then the IPEA can really help. What operator has an eigenvalue related to the enery eigenvalue of the hydrogen molecule? Well, that could be the controlled-$e^{-iHt}$ gate (our $U = e^{-iHt}$) where $H$ is the Hamiltonian of the hydrogen molecule. This is essentially the unitary transformation we would use to time-evolve a wavefunction subject to the Hamiltonian of molecular hydrogen (recall that $\left|\psi(t + \Delta t)\right> = e^{-iH\Delta t}\left|\psi(t)\right>$). Since $H$, the Hamiltonian of the hydrogen molecule, has energy eigenstates that correspond to energy eigenvalues (the ground state energy, first-excited energy for instance), the eigenvalues of the exponential of $H$ will be the exponentials of the energy eigenvalues (for the same eigenstates). More specifically, if $\left|\psi\right>$ is an eigenvector of $H$ corresponding to the eigenvalue $\alpha$, $e^{-iHt}$ satisfies $e^{-iHt}\left|\psi\right>=e^{-i\alpha t}\left|\psi\right>$.

Okay, great! We now know qualitatively what this transformation means from a quantum-physics perspective, and more relevantly, what eigenvalues to expect (just the energy eigenvalues of hydrogen exponentiated). Therefore, we know value of $\phi$ to expect. $$2i\pi\phi = -iEt \implies \phi = -\frac{Et}{2\pi}$$ We will obtain $\phi$ using the IPEA, and we will obviously know how much time we give the system to evolve, thereby obtaining $t$. If you don't know what 2 and $\pi$ are, then I suggest you stop reading right now...



<br>
<br>
<br>
<br>



(just kidding, here you go - https://science.howstuffworks.com/math-concepts/pi.htm#:~:text=The%20definition%20of%20pi%20is,circumference%20divided%20by%20its%20diameter.&text=Divide%20the%20circumference%20of%20the,diameter%20%E2%80%94%20you%20get%20the%20point

https://www.youtube.com/watch?v=BUZaPCLJA3c)

Okay, so we have now understood how we are going to use the IPEA to arrive at the solution. We have our bird's-eye view of the problem. Ready to get into the details? Let's get down into the mud! We now have to worry about how precisely we are going to represent this problem on a quantum circuit. Instead of explaining it here, why don't I show you? Let's get right into it ...

First, we import every library/module that will be needed.

In [3]:
from qiskit import *
#import qiskit.aqua as aqua
#from qiskit.quantum_info import Pauli
#from qiskit.aqua.operators.primitive_ops import PauliOp
from qiskit.circuit.library import PhaseEstimation
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
%matplotlib inline

import pyscf
from pyscf import gto, scf, cc

from matplotlib import pyplot as plt 
import numpy as np

from qiskit_ionq import IonQProvider
provider = IonQProvider("sUkOPlNLf6vamkFMB3SuHR2qYR5cLaMn")

from qiskit.providers.jobstatus import JobStatus
import time as timer

What is this `g` dictionary? Well, in essence, here is how we are going to write our Hamiltonian - 
<br>
<br>
    $H = $ `g["I"]`$I$ + `g["Z0"]`$Z_0$ + `g["Z1"]`$Z_1$ + `g["Z0Z1"]`$Z_0Z_1$ + `g["X0X1"]`$X_0X_1$ + `g["Y0Y1"]`$Y_0Y_1$
    
That's where this `g` dictionary comes in

In [4]:
g = {"I": -0.4804, "Z0": 0.3435, "Z1": -0.4347, "Z0Z1": 0.5716, "X0X1": 0.0910, "Y0Y1": 0.0910}

Now, we write the function `buildControlledHam`. As the name suggests, it creates our Hamiltonian-based gate and applies it in that special way I alluded to earlier in the basic summary of the IPEA. An even simpler way of describing this function would be that this function performs one iteration of the IPEA. So in one iteration, the function `buildControlledHam` - 

- initialises the circuit. In every iteration, the auxiliary qubit will be initialized in the state $\left |+\right> = \frac{1}{\sqrt{2}}(\left|0\right> + \left|1\right>)$, and another qubit will be initialised in the state $\left|\psi\right>$ (this is the eigenvector of $U$ from above; we are using $\left|01\right>$. It turns out that this ket isn't exactly an energy eigenstate. We are using it because it is a state that comes very close to being an energy eigenstate and for the purpose of understanding this notebook, you can treat it like an energy eigenstate).

<br>

- then applies the C$e^{-iHt}$ (controlled-$e^{-iHt}$) gate to the circuit. 

    The C$e^{-iHt}$ gate functions exactly like a CNOT gate - the only difference is that if the control qubit is in state $\left|1\right>$, the $e^{-iHt}$ gate will be applied to the target qubit instead of the $X$ gate.

<br>

- then performs a phase correction. Recall that we need this phase correction to remove these unwanted phases and create the state to which only a measurement in the x-basis is necessary to complete the iteration.

<br>

- and finally, does an inverse Quantum Fourier Transform (QFT), which is nothing but a measurement in the Hadamard basis. It then returns the circuit ready for execution on a quantum computer.

In [5]:
"""
  This function implements the controlled exp(-iHt)
  Inputs:
    phase: phase to correct
    weights: Pauli weights
    time: Time
    order_index: Pauli operator index
    iter_num: iteration
    trotter_step: trotter steps

  Output:
    qc: Quantum circuit
"""
def buildControlledHam(phase, time, iter_num):

    # initialize the circuit
    qc = QuantumCircuit(3, 3)

    # Hardmard on ancilla
    qc.h(0)

    # initialize to |01>
    qc.x(2)
    
    # Z0 
    qc.crz(g["Z0"] * time * 2 * 2**iter_num, 0, 2)

    # Y0Y1
    qc.rz(np.pi/2, 1)
    qc.rz(np.pi/2, 2)
    qc.h(1)
    qc.h(2)
    qc.cx(1, 2)
    qc.crz(g["Y0Y1"] * time * 2 * 2**iter_num, 0, 2)
    qc.cx(1, 2)
    qc.h(1)
    qc.h(2)
    qc.rz(-np.pi/2, 1)
    qc.rz(-np.pi/2, 2)
            
    # Z1
    qc.crz(g["Z1"] * time * 2 * 2**iter_num, 0, 1)

    # X0X1
    qc.h(1)
    qc.h(2)
    qc.cx(1, 2)
    qc.crz(g["X0X1"] * time * 2 * 2**iter_num, 0, 2)
    qc.cx(1, 2)
    qc.h(1)
    qc.h(2)

    # phase correction
    qc.rz(phase, 0)

    # inverse QFT
    qc.h(0)

    qc.measure([0, 1, 2],[0, 1, 2])

    return qc


The next function, as the name once again suggests, performs the IPEA algorithm. The above function just performed one iteration, and as you can see in the body of this function, there is a `for` loop in which `buildControlledHam` is called, which implies that one iteration of this `for` loop represents one iteration of the IPEA. The `for` loop iterates `tot_num_iter` times (the input of the function). This tells us that the input `tot_num_iter` of the function signifies the number of iterations in the IPEA algorithm. The `IPEA` function does its iterations, and in each iteration, it is basically just creating the quantum circuit with the ancilla and $\left|01\right>$ initial state, doing what it needs to do (the bullet points above) to the circuit, running the circuit, recovering the result (the $\phi_k$ for that iteration), and appending it to the `bits` list. Once we get the bits, we can use the expression $\phi = \frac{\phi_1}{2} + \frac{\phi_2}{4} + \frac{\phi_3}{8} + ... + \frac{\phi_n}{2^n}$ to find $\phi$. This last part is precisely what the function `eig_from_bits`, defined after `IPEA` below, does.

In [6]:
"""
  This function implements the controlled exp(-iHt)
  Inputs:
    weights: Pauli weights
    time: Time
    order_index: Pauli operator index
    tot_num_iter: number of iterations
    trotter_step: trotter steps

  Output:
    bits: list of measured bits
"""
def IPEA(time, tot_num_iter, backend_id):

    # get backend
    if (backend_id == "qasm_simulator"):
        backend = Aer.get_backend(backend_id)
    else:
        backend = provider.get_backend("ionq_qpu")

    # bits
    bits = []

    # phase correction
    phase = 0.0

    # loop over iterations 
    for i in range(tot_num_iter-1, -1, -1):

        # construct the circuit
        qc = buildControlledHam(phase, time, i)
        # run the circuit
        job = execute(qc, backend)
        
        if (backend_id == "ionq_qpu"):

            # Check if job is done
            while job.status() is not JobStatus.DONE:
                print("Job status is", job.status() )
                timer.sleep(60)

            # once we break out of that while loop, we know our job is finished
            print("Job status is", job.status() )

        # get result
        result = job.result()

        # get current bit
        this_bit = int(max(result.get_counts(), key=result.get_counts().get)[-1])
        
        bits.append(this_bit)

        # update phase correction
        phase /= 2
        phase -= (2 * np.pi * this_bit / 4.0)

    return bits

If you have made it this far, then you are doing very well! The IPEA algorithm is complicated because it has many moving parts to it, so take your time and make sure you can trace everything. Good job! 

The final function that we will have to define is `eig_from_bits`, which, as mentioned above, will take in the list of $\phi_k$'s that the above function will return, and return the eigenvalue associated with $\left|\psi\right>$

In [7]:
"""
  This function that computes eigenvalues from bit list
  Inputs:
    bits: list of measured bits
    time: Time

  Output:
    eig: eigenvalue
"""
def eig_from_bits(bits, time):

    eig = 0.

    m = len(bits)

    # loop over all bits
    for k in range(len(bits)):
        eig += bits[k] / (2**(m-k))

    eig *= -2*np.pi
    eig /= time

    return eig

The IPEA is done! But notice that in the cell above, you get the phase, but not the actual energy. The function below just takes in physical quantities like bond distance, and the result of the IPEA, to compute the actual energy value.

In [8]:
"""
  This function that performs classical post-processing
  Inputs:
    eig:     eigenvalue
    weights: Pauli operator weights
    R:       Bond distance

  Output:
    energy: total energy
"""
def post_process(eig, weights, R):

    # initialize energy
    energy = eig

    # Z0Z1 contribution
    energy -= weights["Z0Z1"]

    # I contribution
    energy += weights["I"]

    # Nuclear Repulsion ( assume R is in Angstrom )
    energy += 1.0/ (R * 1.88973)

    # return energy 
    return energy


Okay, now we have all our functions defined to perform the IPEA and retrieve the energy associated with the oinergoinergoinergoienrgoienrg of the Hydrogen molecule. First, we will use a Quantum Chemistry library known as `pyscf` to calculate the value that we expect our design to return.

In [9]:
# backend
backend_id = "qasm_simulator"

# Bond Length
R = 0.75

# time 
t = 0.74

# number of iteration
max_num_iter = 8

# get exact energy 
# This function initialises a molecule

mol = pyscf.M(
    atom = 'H 0 0 0; H 0 0 0.75',  # in Angstrom
    basis = 'sto-6g',
    symmetry = False,
)
myhf = mol.RHF().run()

# create an FCI solver based on the SCF object
#
cisolver = pyscf.fci.FCI(myhf)
print('E(FCI) = %.12f' % cisolver.kernel()[0])


converged SCF energy = -1.1247307455369
E(FCI) = -1.145741671075


Now, let us check to see whether we can get the energy to converge to that value using our implementation. 

In [10]:
error_list = []

# perform IPEA
for i in range(1, max_num_iter + 1):
    bits = IPEA(t, i, backend_id)

    # re-construct phase
    eig = eig_from_bits(bits, t)
    print(eig, 'eig')

    # re-construct energy
    eng = post_process(eig, g, R)
                
    print("Total Energy is %.7f for R = %.2f, t = %.3f with %d iterations" % (eng, R, t, i) )

-0.0 eig
Total Energy is -0.3464318 for R = 0.75, t = 0.740 with 1 iterations
-0.0 eig
Total Energy is -0.3464318 for R = 0.75, t = 0.740 with 2 iterations
-1.0613488694560111 eig
Total Energy is -1.4077807 for R = 0.75, t = 0.740 with 3 iterations
-0.5306744347280056 eig
Total Energy is -0.8771063 for R = 0.75, t = 0.740 with 4 iterations
-0.7960116520920084 eig
Total Energy is -1.1424435 for R = 0.75, t = 0.740 with 5 iterations
-0.7960116520920084 eig
Total Energy is -1.1424435 for R = 0.75, t = 0.740 with 6 iterations
-0.7960116520920084 eig
Total Energy is -1.1424435 for R = 0.75, t = 0.740 with 7 iterations
-0.7960116520920084 eig
Total Energy is -1.1424435 for R = 0.75, t = 0.740 with 8 iterations


### Now, let's try the same thing on actual IonQ hardware!

In [11]:
backend_id = "qpu"
error_list = []

# perform IPEA
for i in range(1, max_num_iter + 1):
    bits = IPEA(t, i, backend_id)

    # re-construct phase
    eig = eig_from_bits(bits, t)

    # re-construct energy
    eng = post_process(eig, g, R)
                
    print("Total Energy is %.7f for R = %.2f, t = %.3f with %d iterations" % (eng, R, t, i) )

/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/qiskit/execute_function.py:343: UserWarning: Option parameter_binds is not used by this backend
  job = backend.run(experiments, **run_kwargs)
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably

Total Energy is -0.3464318 for R = 0.75, t = 0.740 with 1 iterations


/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). Thi

Total Energy is -0.3464318 for R = 0.75, t = 0.740 with 2 iterations


/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). Thi

Total Energy is -1.4077807 for R = 0.75, t = 0.740 with 3 iterations


/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). Thi

Total Energy is -0.8771063 for R = 0.75, t = 0.740 with 4 iterations


/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). This will probably lead to SSL verification errors
  warnings.warn(
/Users/abhay/opt/anaconda3/lib/python3.8/site-packages/urllib3/connection.py:379: SystemTimeWarning: System time is way off (before 2020-07-01). Thi

KeyboardInterrupt: 